In [ ]:
#include "SMCEDM_Analysis.h"
#include "progressbar.hpp"


double legendre_l(int l, double x)
{
	if(l==0) return 1;
	else if(l==1) return x;
	else if(l==2) return 0.5*(3*x*x-1);
	else return 0.5*(5*x*x*x-3*x);
}

double cos_omega_ij(jets j1, jets j2)
{
	//"calculates the 3D angle between j1 and j2"
	TVector3 p1 = j1.get4P().Vect();
	TVector3 p2 = j2.get4P().Vect();
	return p1.Dot(p2)/(p1.Mag()*p2.Mag());
}

void SMCEDM_Analysis()
{
	int nthreads = 24;
	ROOT::EnableImplicitMT(nthreads);

	//auto file = new TFile("tag_1_delphes_events.root");
	auto file = new TFile("/mnt/harddisk2/wjets/WJets_600to800/Events/delphes.root");
	//auto file = new TFile("/mnt/harddisk1/ttbar/dtG_4/Events/run_02/delphes.root");
	TTreeReader myReader("Delphes", file);

	auto outfile = new TFile("dtG_50_flat.root", "RECREATE");

	// Create the TTree and branches
    TTree *outtree = new TTree("outtree", "outtree");

    int   n_jets, n_leptons, n_bjets;
    float scalar_ht, jet_pt[4], met, met_phi, sphericity, fox_wolfram[4];

    outtree->Branch("br_njets"       , &n_jets,      "n_jets/I");
    outtree->Branch("br_nleptons"    , &n_leptons,   "n_leptons/I");
    outtree->Branch("br_nbjets"      , &n_bjets,     "n_bjets/I");
    outtree->Branch("br_scalar_HT"   , &scalar_ht,   "scalar_ht/F");
    outtree->Branch("br_jet_pt"      , &jet_pt,      "jet_pt[4]/F");
    outtree->Branch("br_MET"         , &met,         "MET/F");
    outtree->Branch("br_MET_Phi"     , &met_phi,     "MET_Phi/F");
    outtree->Branch("br_sphericity"  , &sphericity,  "sphericity/F");
    outtree->Branch("br_Fox_Wolfram" , &fox_wolfram,  "fox_wolfram[4]/F");

	//Jet Definitions
	TTreeReaderValue<int>            rv_Jet_size(myReader, "Jet_size");
	TTreeReaderArray<float>          ra_Jet_pT(myReader,   "Jet.PT");
	TTreeReaderArray<float>          ra_Jet_Eta(myReader,  "Jet.Eta");
	TTreeReaderArray<float>          ra_Jet_Phi(myReader,  "Jet.Phi");
	TTreeReaderArray<float>          ra_Jet_Mass(myReader, "Jet.Mass");
	TTreeReaderArray<unsigned int>   ra_Jet_BTag(myReader, "Jet.BTag");

	//Electron Definitions
	TTreeReaderValue<int>            rv_Electron_size(myReader,  "Electron_size");
	TTreeReaderArray<float>          ra_Electron_pT(myReader,    "Electron.PT");
	TTreeReaderArray<float>          ra_Electron_Eta(myReader,    "Electron.Eta");
	TTreeReaderArray<float>          ra_Electron_Phi(myReader,    "Electron.Phi");
	TTreeReaderArray<float>          ra_Electron_Energy(myReader,   "Electron.T");
	TTreeReaderArray<int>            ra_Electron_Charge(myReader, "Electron.Charge");

	//Muon Definitions
	TTreeReaderValue<int>            rv_Muon_size(myReader,  "Muon_size");
	TTreeReaderArray<float>          ra_Muon_pT(myReader,    "Muon.PT");
	TTreeReaderArray<float>          ra_Muon_Eta(myReader,    "Muon.Eta");
	TTreeReaderArray<float>          ra_Muon_Phi(myReader,    "Muon.Phi");
	TTreeReaderArray<float>          ra_Muon_Energy(myReader,   "Muon.T");
	TTreeReaderArray<int>            ra_Muon_Charge(myReader, "Muon.Charge");

	//MET Definitions
	TTreeReaderArray<float>          ra_MissingET_MET(myReader,   "MissingET.MET");
	TTreeReaderArray<float>          ra_MissingET_Phi(myReader,   "MissingET.Phi");

	//HT Definitions
	TTreeReaderArray<float>          ra_scalar_HT(myReader,   "ScalarHT.HT");

	// Histogram Definitions
	auto histo_bjet_pT      = new TH1F("histo_bjet_pT", "histo_bjet_pT", 200, 0, 2000);
	auto histo_light_jet_pT = new TH1F("histo_light_jet_pT", "histo_light_jet_pT", 200, 0, 2000);
	auto histo_had_top_mass = new TH1F("histo_had_top_mass", "histo_had_top_mass", 1000, 0, 1000);
	auto histo_had_top_eta  = new TH1F("histo_had_top_eta", "histo_had_top_eta", 100, -5, 5);
	auto histo_had_top_phi  = new TH1F("histo_had_top_phi", "histo_had_top_phi", 30, -3.14, 3.14);

	auto n_events = myReader.GetEntries(1);
	std::cout << "n_events:" << n_events << std::endl;

	int n_btag;
	int n_btag_cut = 0;	
	int n_light_jet;
	int n_light_jet_cut = 0;
	int n_lepton_cut = 0;
	int n_MET_cut  = 0;
	int n_events_passed = 0;
	float MET_threshold = 0.0;

	// initialize the progress bar
    const int limit = myReader.GetEntries(1);
    ProgressBar progressBar(limit, 70);

	// Loop over Events
	for (int i_event = 0; i_event < n_events; ++i_event)
	{
		++progressBar;
        // display the bar
        progressBar.display();

		n_btag = 0;

		bool bool_lepton_cut(1), bool_b_jet_cut(1), bool_MET_cut(1), bool_light_jet_cut(1);

		myReader.SetEntry(i_event);

		// put all electrons to a container (vector of leptons object)
		std::vector<leptons> v_electrons;
		leptons dummy_electron;
		
		//******* count the number of negative and positive electrons *******//
		for (int i_electron = 0; i_electron < *rv_Electron_size; ++i_electron)
		{
			dummy_electron.setPt(ra_Electron_pT.At(i_electron));
			dummy_electron.setEta(ra_Electron_Eta.At(i_electron));
			dummy_electron.setPhi(ra_Electron_Phi.At(i_electron));
			dummy_electron.setM(0.511/1000);
			dummy_electron.setQ(ra_Electron_Charge.At(i_electron));
			dummy_electron.setPDGID(11);

			v_electrons.push_back(dummy_electron);
		}
		
		// put all muons to a container (vector of leptons object)
		std::vector<leptons> v_muons;
		leptons dummy_muon;
		//******* count the number of negative and positive muons *******//
		for (int i_muon = 0; i_muon < *rv_Muon_size; ++i_muon)
		{
			dummy_muon.setPt(ra_Muon_pT.At(i_muon));
			dummy_muon.setEta(ra_Muon_Eta.At(i_muon));
			dummy_muon.setPhi(ra_Muon_Phi.At(i_muon));
			dummy_muon.setM(105/1000);
			dummy_muon.setQ(ra_Muon_Charge.At(i_muon));
			dummy_muon.setPDGID(13);

			v_muons.push_back(dummy_muon);
		}

		// put all jets to a container (vector of jets object)
		std::vector<jets> v_jets;
		jets dummy_jet;
		//******* count the number of negative and positive muons *******//
		for (int i_jet = 0; i_jet < *rv_Jet_size; ++i_jet)
		{
			dummy_jet.setPt(ra_Jet_pT.At(i_jet));
			dummy_jet.setEta(ra_Jet_Eta.At(i_jet));
			dummy_jet.setPhi(ra_Jet_Phi.At(i_jet));
			dummy_jet.setM(ra_Jet_Mass.At(i_jet));
			dummy_jet.setBTag(ra_Jet_BTag.At(i_jet));

			v_jets.push_back(dummy_jet);
		}

		if(v_electrons.size() + v_muons.size() < 1)
		{
			bool_lepton_cut = 0; 
			n_lepton_cut++;
		}
		 
		std::vector<jets> v_light_jets;
		std::vector<jets> v_b_jets;

		for (int i_jet = 0; i_jet < v_jets.size(); ++i_jet)
		{
			if(v_jets.at(i_jet).getBTag()!=1)
			{
				v_light_jets.push_back(v_jets.at(i_jet));
			}
			else
			{
				v_b_jets.push_back(v_jets.at(i_jet));
			}
		}


		n_btag = *rv_Jet_size - v_light_jets.size();

		if(v_light_jets.size() < 2)
		{
			bool_light_jet_cut = 0;
			n_light_jet_cut++;
		}

		if(n_btag < 2)
		{
			bool_b_jet_cut = 0;
			n_btag_cut++;
		}
		
		//******* skip the event if MET < threshold *******//
		//std::cout << "MET:" << ra_MissingET_MET[0] << std::endl;
		if (ra_MissingET_MET[0] < MET_threshold)
		{
			n_MET_cut++;
			bool_MET_cut = 0;
			continue;
		}

	
		if(bool_lepton_cut==0 || bool_MET_cut==0 || bool_light_jet_cut==0 || bool_b_jet_cut==0)
		{
			continue;
		}

		std::vector<candidate> v_W_candidates;
		
		for (int i_jet = 0; i_jet < v_light_jets.size(); ++i_jet)
		{
			for (int j_jet = 0; j_jet < v_light_jets.size(); ++j_jet)
			{
				if(i_jet < j_jet)
				{
					candidate dummy_candidate(v_light_jets.at(i_jet), v_light_jets.at(j_jet));
					v_W_candidates.push_back(dummy_candidate);
				}
			}
		}

		std::vector<candidate> v_had_top_candidates;

		for (int i_w_candidate = 0; i_w_candidate < v_W_candidates.size(); ++i_w_candidate)
		{
			for (int j_bjet = 0; j_bjet < v_b_jets.size(); ++j_bjet)
			{
				candidate dummy_candidate(v_W_candidates.at(i_w_candidate), v_b_jets.at(j_bjet));
				v_had_top_candidates.push_back(dummy_candidate);
			}
		}

		std::sort(v_had_top_candidates.begin(), v_had_top_candidates.end(),[](candidate first, candidate second) {return abs(first.getM()- 173.) < abs(second.getM()- 173.);});

		histo_had_top_mass->Fill(v_had_top_candidates.at(0).getM());
		histo_had_top_eta->Fill(v_had_top_candidates.at(0).get4P().Eta());
		histo_had_top_phi->Fill(v_had_top_candidates.at(0).get4P().Phi());

		n_jets    = v_jets.size();
		n_bjets   = v_b_jets.size();
		n_leptons = v_electrons.size() + v_muons.size();
		scalar_ht = ra_scalar_HT[0];

		for (int i_jet = 0; i_jet < 4; ++i_jet)
		{
			jet_pt[i_jet] = v_jets.at(i_jet).getPt();
		}


		//construct the sphericity tensor
		double Sxx(0), Sxy(0), Sxz(0);
		double Syx(0), Syy(0), Syz(0);
		double Szx(0), Szy(0), Szz(0);

		double sump2(0), sumE(0);

		for (int i_jet = 0; i_jet < v_jets.size(); ++i_jet)
		{
			Sxx += v_jets.at(i_jet).get4P().Px()*v_jets.at(i_jet).get4P().Px();
			Sxy += v_jets.at(i_jet).get4P().Px()*v_jets.at(i_jet).get4P().Py();
			Sxz += v_jets.at(i_jet).get4P().Px()*v_jets.at(i_jet).get4P().Pz();

			Syy += v_jets.at(i_jet).get4P().Py()*v_jets.at(i_jet).get4P().Py();
			Syz += v_jets.at(i_jet).get4P().Py()*v_jets.at(i_jet).get4P().Pz();

			Szz += v_jets.at(i_jet).get4P().Pz()*v_jets.at(i_jet).get4P().Pz();

			sump2 += pow(v_jets.at(i_jet).get4P().E(),2) - pow(v_jets.at(i_jet).get4P().M(),2);
			sumE  += v_jets.at(i_jet).get4P().E();
		}

		if(sump2 > 0)
		{
			Sxx = Sxx/sump2;
			Sxy = Sxy/sump2;
			Sxz = Sxz/sump2;

			Syy = Syy/sump2;
			Syz = Syz/sump2;

			Szz = Szz/sump2;
		}

		TMatrixDSym S(3);
		S(0, 0) = Sxx;
		S(0, 1) = Sxy;
		S(0, 2) = Sxz;
		S(1, 0) = Sxy;
		S(1, 1) = Syy;
		S(1, 2) = Syz;
		S(2, 0) = Sxz;
		S(2, 1) = Syz;
		S(2, 2) = Szz;

		//---- compute sphericity -----------------------
		TMatrixDSymEigen TEigen(S);
		TVectorD eigenValues(TEigen.GetEigenValues());
		sphericity = 1.5 * (eigenValues(1) + eigenValues(2));
		/////////////////////////////////

		//---- compute Fox-Wolfram Moment-----------------------
		double px1, py1, pz1, px2, py2, pz2, p_i, p_j;
		double E_vis2 = sumE*sumE;
		double cos_omega;

		// reset the fox_wolfram moments to 0 for each event
		fox_wolfram[0] = 0.;
		fox_wolfram[1] = 0.;
		fox_wolfram[2] = 0.;
		fox_wolfram[3] = 0.;
		//std::cout << "number of jets:" << v_jets.size() << std::endl;
		
		for (int i_jet = 0; i_jet < v_jets.size(); ++i_jet)
		{
			for (int j_jet = 0; j_jet < v_jets.size(); ++j_jet)
			{
				if(i_jet <= j_jet)
				{
					cos_omega = cos_omega_ij(v_jets.at(i_jet), v_jets.at(j_jet));
					p_i = v_jets.at(i_jet).get4P().P();
					p_j = v_jets.at(j_jet).get4P().P();

					for (int l = 0; l < 4; ++l)
					{
						fox_wolfram[l] += (p_i*p_j)*legendre_l(l, cos_omega);
					}
				}
			}
		}
		/////////////////////////////////

		met     = ra_MissingET_MET[0];
		met_phi = ra_MissingET_Phi[0];

		outtree->Fill();
		n_events_passed++;
	}

	outfile->cd();
	outtree->Write();

	std::cout << "n_light_jet_cut: " << n_light_jet_cut << std::endl;
	std::cout << "n_btag_cut: " << n_btag_cut << std::endl;
	std::cout << "n_lepton_cut: " << n_lepton_cut << std::endl;
	std::cout << "n_MET_cut: " << n_MET_cut << std::endl;
	std::cout << "n_events_passed/n_events: " << n_events_passed << "/" << n_events << std::endl;

	gROOT->LoadMacro("tdrstyle.C");
        gROOT->ProcessLine("setTDRStyle();");

	TCanvas *canvas1 = new TCanvas("canvas1", "canvas1", 600, 600); 
	histo_had_top_mass->Draw("HIST");

	TCanvas *canvas2 = new TCanvas("canvas2", "canvas2", 600, 600);
	histo_had_top_eta->Draw("HIST");

	TCanvas *canvas3 = new TCanvas("canvas3", "canvas3", 600, 600);
	histo_had_top_phi->Draw("HIST");
}
